# Assistant APIs with Code Interpreter for Audit

In [1]:
import os
import json
import requests
import time
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv() 

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
  api_key= os.getenv("AZURE_OPENAI_API_KEY"),
  api_version="2024-05-01-preview"
)




In [2]:
file = client.files.create(file=open("data.csv", "rb"), purpose="assistants")

In [3]:
metaprompt = """"
You are a Risk Detector assistant. You are specialized in running code against provided files to identify anomalies and outliers. 
Whenever you are asked about outliers, always return the row of the outlier, including columns and values, and save the table with outlier in a csv format.

"""

In [4]:
assistant = client.beta.assistants.create(
  model="gpt-4o", # replace with model deployment name.
  instructions= metaprompt,
  tools=[{"type":"code_interpreter"}],
  tool_resources={"code_interpreter":{"file_ids":[file.id]}},
  temperature=1,
  top_p=1
)

In [5]:
# Create a thread
thread = client.beta.threads.create()

# Add a user question to the thread
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="are there any outliers?" # Replace this with your prompt
)


# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

# Looping until the run completes or fails
while run.status in ['queued', 'in_progress', 'cancelling']:
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages.data[0].content[0].text.value)
elif run.status == 'requires_action':
  # the assistant requires calling some functions
  # and submit the tool outputs back to the run
  pass
else:
  print(run.status)


It appears that there are no outliers detected in the data based on the IQR method.

If you have any further analysis or a different method in mind for detecting outliers, please let me know how you would like to proceed.


In [6]:
def get_file_ids_from_thread(client, thread_id):
    """
    Extracts file_ids from the attachments of messages in a thread, if attachments are not empty.

    :param client: The client object used to interact with the API.
    :param thread_id: The ID of the thread to extract messages from.
    :return: A list of file_ids extracted from the attachments of the messages.
    """
    # Get the list of messages from the thread
    messages = [i for i in client.beta.threads.messages.list(thread_id=thread_id)]

    # Extract file_ids from each message's attachments if attachments are not empty
    file_ids = [
        attachment.file_id
        for message in messages
        if message.attachments
        for attachment in message.attachments
    ]
    
    return file_ids

# Example usage
file_ids = get_file_ids_from_thread(client, thread_id='thread_12yh3C890jvQYzTsutTUc4rh')
print(file_ids)  # Output will be a list of file_ids from all messages with attachments


['assistant-o6QzcwtUgZJm7aj8NWXsdwVD']


In [7]:
def write_file_to_temp_dir(file_id, output_path):
    file_data = client.files.content(file_id)
    file_data_bytes = file_data.read()
    with open(output_path, "wb") as file:
        file.write(file_data_bytes)


some_file_id = file_ids[0]
write_file_to_temp_dir(some_file_id, 'outliers.csv')

# Testing bulk queries

In [9]:
import time

def run_queries(queries, client, assistant_id):
    results = []
    
    for query in queries:
        # Create a thread for each query
        thread = client.beta.threads.create()

        # Add the user query to the thread
        message = client.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content=query  # Use the current query from the list
        )

        # Run the assistant with the query
        run = client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=assistant_id
        )

        # Looping until the run completes or fails
        while run.status in ['queued', 'in_progress', 'cancelling']:
            time.sleep(1)
            run = client.beta.threads.runs.retrieve(
                thread_id=thread.id,
                run_id=run.id
            )

        # Collect the result once the run is completed
        if run.status == 'completed':
            messages = client.beta.threads.messages.list(
                thread_id=thread.id
            )
            results.append(messages.data[0].content[0].text.value)
        elif run.status == 'requires_action':
            # Handle any required actions if necessary
            results.append("Assistant requires action for query: " + query)
        else:
            # Store the status for failed queries
            results.append(f"Query failed: {run.status} for query: {query}")

    return results


# List of queries to run
queries = [
    "Are there any outliers?",
    "Can you summarize the data trends?",
    "What anomalies do you detect?"
]

# Call the function with the list of queries
results = run_queries(queries, client, assistant.id)

# Output the results for each query
for i, result in enumerate(results):
    print(f"Result for query {i+1}: {result}")


Result for query 1: I have identified the outliers in the dataset. Here are the rows that contain outliers:

| ID | Company     | Revenue (USD) | Expenses (USD) | Profit (USD) |
|----|-------------|---------------|----------------|--------------|
| 6  | Zeta Corp   | 1,500,000     | 1,200,000      | 300,000      |
| 18 | Sigma Corp  | 1,800,000     | 1,500,000      | 300,000      |
| 25 | Alpha2 Corp | 2,500,000     | 2,000,000      | 500,000      |
| 30 | Zeta2 Corp  | 1,200,000     | 1,000,000      | 200,000      |
| 34 | Kappa2 Corp | 2,200,000     | 1,700,000      | 500,000      |
| 40 | Pi2 Ltd.    | 3,000,000     | 2,500,000      | 500,000      |

The file containing these outliers has been saved at: [outliers.csv](sandbox:/mnt/data/outliers.csv)
Result for query 2: Here is a summary of the data trends based on the calculated statistics:

1. **Revenue (USD)**:
   - Mean (average) revenue: $476,625
   - Standard deviation (spread): $726,632.9
   - Minimum revenue: $45,000
   - Max

# Display images

In [ ]:
from PIL import Image
import io

# Assuming response_content is the binary content you're getting from the OpenAI API
response_content = client.files.content(messages.data[0].content[0].image_file.file_id)

# Convert the binary response content to an image
image_bytes = response_content.content
image = Image.open(io.BytesIO(image_bytes))

# Display the image
image.show()




In [57]:
messages

SyncCursorPage[Message](data=[Message(id='msg_WPUgNixVlcYTmKw4W3pKZ3Ce', assistant_id='asst_4aatBVInAyCOhUfzWA5M8yZD', attachments=[Attachment(file_id='assistant-o6QzcwtUgZJm7aj8NWXsdwVD', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FilePathAnnotation(end_index=147, file_path=FilePath(file_id='assistant-o6QzcwtUgZJm7aj8NWXsdwVD'), start_index=117, text='sandbox:/mnt/data/outliers.csv', type='file_path')], value='The outliers have been saved to a CSV file. You can download the file using the link below:\n\n[Download outliers.csv](sandbox:/mnt/data/outliers.csv)'), type='text')], created_at=1729577434, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_7NXbrQMJPp73SXurSWDzbfSo', status=None, thread_id='thread_12yh3C890jvQYzTsutTUc4rh'), Message(id='msg_jUNoPpyPnv1iNy5ksR6Faayr', assistant_id='asst_4aatBVInAyCOhUfzWA5M8yZD', attachments=[], complete

In [61]:
run_steps = client.beta.threads.runs.steps.list(
  thread_id=thread.id,
  run_id=run.id
)



SyncCursorPage[RunStep](data=[RunStep(id='step_AbDJI5I1aABzai99pxuAH9Cc', assistant_id='asst_4aatBVInAyCOhUfzWA5M8yZD', cancelled_at=None, completed_at=1729577435, created_at=1729577434, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_7NXbrQMJPp73SXurSWDzbfSo', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_WPUgNixVlcYTmKw4W3pKZ3Ce'), type='message_creation'), thread_id='thread_12yh3C890jvQYzTsutTUc4rh', type='message_creation', usage=Usage(completion_tokens=37, prompt_tokens=1645, total_tokens=1682), expires_at=None), RunStep(id='step_AYnkm56sje4mZkMtUofdQiTD', assistant_id='asst_4aatBVInAyCOhUfzWA5M8yZD', cancelled_at=None, completed_at=1729577434, created_at=1729577432, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_7NXbrQMJPp73SXurSWDzbfSo', status='completed', step_details=ToolCallsStepDetails(tool_calls=[CodeI

In [81]:
print(run_steps.data[3].step_details.tool_calls[0].code_interpreter.input)

# Convert columns to numeric
data['Revenue (USD)'] = data['Revenue (USD)'].str.replace(',', '').astype(float)
data['Expenses (USD)'] = data['Expenses (USD)'].str.replace(',', '').astype(float)
data['Profit (USD)'] = data['Profit (USD)'].str.replace(',', '').astype(float)

def detect_outliers_iqr(df):
    outliers = pd.DataFrame(columns=df.columns)
    for column in df.select_dtypes(include=[float, int]).columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        column_outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
        outliers = pd.concat([outliers, column_outliers])

    return outliers

outliers = detect_outliers_iqr(data)

outliers
